In [99]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_similarity(s1, s2):    
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([s1, s2])
    similarity = cosine_similarity(vectors[0], vectors[1])
    
    return similarity[0][0]

In [101]:
mapping_path = "db/mapping/export/export_mapping.csv"
data_path = "db/tw/data/export/tw_export_m.csv"

mapping = pd.read_csv(mapping_path, index_col=None)
data = pd.read_csv(data_path, index_col=[0])


In [143]:
index = 0
keep_list = [1, 5, 3]
mapping_list = mapping.iloc[[index]].T[index]

if mapping_list[2] == "Value":
    mapping_list = mapping_list[[1, 5, 3]].dropna()
else:
    mapping_list = mapping_list[[1, 5, 2, 3]].dropna()
    
if "Total" in mapping_list:
    mapping_list.remove("Total")
    
mapping_string = ", ".join(mapping_list)
mapping_string

'Exports, USD'

In [144]:
fuzzscore_list = []
cosscore_list = []


data_columns = data.columns

for i in mapping_list:
    data_columns = data_columns[data_columns.str.lower().str.contains(i.lower())]

data_columns_revised = data_columns.str.replace(", All", "")
    
for i in data_columns_revised:
    fuzzscore = fuzz.token_sort_ratio(i, mapping_string) / 100
    cosscore = get_similarity(i, mapping_string)
    
    fuzzscore_list.append(fuzzscore)
    cosscore_list.append(cosscore)
    

In [176]:
result = pd.DataFrame({"dataname": data_columns.tolist(), "fuzzscore": fuzzscore_list, "cosscore": cosscore_list}).sort_values(by=["cosscore", "fuzzscore"], ascending=False)
result = result.loc[result['fuzzscore'] > 0.6]
result['overallscore'] = (result['fuzzscore'] + result['cosscore']) / 2
result['mappingname'] = mapping_string
result['mappingindex'] = index
result['matchinglength'] = len(result)

result.index = range(len(all_result), len(all_result) + len(result))

In [275]:
from tqdm import tqdm
all_result = pd.DataFrame()

for index in tqdm(mapping.index, desc='Processing mapping', unit='mapping'):
    mapping_list = mapping.iloc[[index]].T[index]
    if mapping_list[0] != "Monthly":
        continue

    if mapping_list[2] == "Value":
        mapping_list = mapping_list[[1, 5, 3]].dropna()
    else:
        mapping_list = mapping_list[[1, 5, 2, 3]].dropna()

    if "Total" in mapping_list:
        mapping_list.remove("Total")

    mapping_string = ", ".join(mapping_list)

    data_columns = data.columns

    for i in mapping_list:
        data_columns = data_columns[data_columns.str.lower().str.contains(i.lower())]
    

    data_columns_revised = data_columns.str.replace(", All", "")

    fuzzscore_list = []
    cosscore_list = []
    for i in data_columns_revised:
        fuzzscore = fuzz.token_sort_ratio(i, mapping_string) / 100
        cosscore = get_similarity(i, mapping_string)

        fuzzscore_list.append(fuzzscore)
        cosscore_list.append(cosscore)

    result = pd.DataFrame({"dataname": data_columns.tolist(), "fuzzscore": fuzzscore_list, "cosscore": cosscore_list}).sort_values(by=["cosscore", "fuzzscore"], ascending=False)
    result = result.loc[(result['fuzzscore'] > 0.6) & (result['cosscore'] > 0.5)]
        
    result['overallscore'] = (result['fuzzscore'] + result['cosscore']) / 2
    result['mappingname'] = mapping_string
    result['mappingindex'] = index
    result['matchinglength'] = len(result)

    result.index = range(len(all_result), len(all_result) + len(result))

    all_result = pd.concat([all_result, result], axis=0)


Processing mapping: 100%|█████████████| 5346/5346 [00:11<00:00, 474.18mapping/s]


In [276]:
duplicated_dataname_u = all_result[all_result['dataname'].duplicated()]['dataname'].unique()

for duplicate in duplicated_dataname_u:
    duplicate_df = all_result.loc[all_result['dataname'] == duplicate].sort_values(by="overallscore", ascending=False)
    all_result.drop(duplicate_df.index[1:], inplace=True)

In [277]:
duplicated_mappingname_u = all_result[all_result['mappingname'].duplicated()]['mappingname'].unique()

for duplicate in duplicated_mappingname_u:
    duplicate_df = all_result.loc[all_result['mappingname'] == duplicate].sort_values(by="overallscore", ascending=False)
    all_result.drop(duplicate_df.index[1:], inplace=True)

In [278]:
all_result

,dataname,fuzzscore,cosscore,overallscore,mappingname,mappingindex,matchinglength
0,"Exports, All, USD",1.00,1.000000,1.000000,"Exports, USD",0,31
31,"Exports, Semiconductors HS 776, USD",0.88,0.656973,0.768486,"Exports, Semiconductors, USD",1,5
36,"Exports, China and Hong Kong, USD",0.71,0.579739,0.644869,"Exports, China, USD",5,3
39,"Exports, DM excl Japan, USD",0.81,0.656973,0.733486,"Exports, Japan, USD",6,5
44,"Exports, EU, USD",1.00,1.000000,1.000000,"Exports, EU, USD",9,6
50,"Exports, US, USD",1.00,1.000000,1.000000,"Exports, US, USD",10,12
62,"Exports, All, LCU",1.00,1.000000,1.000000,"Exports, LCU",11,30
92,"Exports, Semiconductors HS 776, LCU",0.88,0.656973,0.768486,"Exports, Semiconductors, LCU",12,5
97,"Exports, China and Hong Kong, LCU",0.71,0.579739,0.644869,"Exports, China, LCU",16,3
100,"Exports, DM excl Japan, LCU",0.81,0.656973,0.733486,"Exports, Japan, LCU",17,5


In [264]:
mapping.loc[all_result['mappingindex'], "TW"] = all_result['dataname'].tolist()

In [268]:
mapping.loc[~mapping["TW"].isna()]

,0,1,2,3,4,5,TW
0,Monthly,Exports,Value,USD,Total,NaN,"Exports, All, USD"
1,Monthly,Exports,Value,USD,By Product,Semiconductors,"Exports, Semiconductors HS 776, USD"
5,Monthly,Exports,Value,USD,By Country,China,"Exports, China and Hong Kong, USD"
6,Monthly,Exports,Value,USD,By Country,Japan,"Exports, DM excl Japan, USD"
9,Monthly,Exports,Value,USD,By Country,EU,"Exports, EU, USD"
10,Monthly,Exports,Value,USD,By Country,US,"Exports, US, USD"
11,Monthly,Exports,Value,LCU,Total,NaN,"Exports, All, LCU"
12,Monthly,Exports,Value,LCU,By Product,Semiconductors,"Exports, Semiconductors HS 776, LCU"
16,Monthly,Exports,Value,LCU,By Country,China,"Exports, China and Hong Kong, LCU"
17,Monthly,Exports,Value,LCU,By Country,Japan,"Exports, DM excl Japan, LCU"


In [279]:
len(all_result)

53

In [3]:
import datetime

# Create a datetime object
dt = datetime.datetime(2023, 5, 18, 12, 0, 0)

# Convert datetime to epoch time
epoch_time = dt.timestamp()

print(epoch_time)



1684382400.0
